In [106]:
import urllib2
import json
import pandas as pd

In [94]:
basequery = 'http://harrypotter.wikia.com/api.php?action=query&format=json&cmlimit=500'

def create_query(params):
    return basequery+'&' + '&'.join(params)

def create_subcategory_query(cmtitle):
    return create_query(['list=categorymembers', 'cmtitle='+cmtitle, 'cmtype=subcat'])

def create_page_query(cmtitle):
    return create_query(['list=categorymembers', 'cmtitle='+cmtitle, 'cmtype=page'])

In [90]:
def get_sub_categories(title):
    try:
        query = create_subcategory_query(title)
        query = query.replace(' ', '%20')
        response =  urllib2.urlopen(query)
    except Exception as e:
        print(e)
        print(query)
        return [title]
    response_dict = json.loads(response.read())
    if 'query' not in response_dict:
        return [title]
    return [title] + [c for d in response_dict['query']['categorymembers'] \
                      for c in get_sub_categories(d['title'])]

In [130]:
houses = ['Category:Gryffindors', 
          'Category:Hufflepuffs', 
          'Category:Ravenclaws', 
          'Category:Slytherins', 
          'Category:Unknown_House']

In [131]:
pages = []
count = 0

for house in houses:
    cs = get_sub_categories(house)
    for c in cs:
        query = create_page_query(c)
        query = query.replace(' ', '%20')
        response =  urllib2.urlopen(query)

        response_dict = json.loads(response.read())
        for p in response_dict['query']['categorymembers']:
            pages.append((p['pageid'], p['title'], house.split(':')[1]))

        print(count)

1
2
3
4
5


In [132]:
df = pd.DataFrame(np.transpose([[p[0] for p in pages], 
                   [p[1] for p in pages], 
                   [p[2] for p in pages]]), 
                  columns=['page_id', 'title', 'House'])

In [135]:
df.head()

,page_id,title,House
0,4982,Template:Gryffindor individual infobox,Gryffindors
1,33349,Astrix Alixan,Gryffindors
2,33353,Filemina Alchin,Gryffindors
3,7018,Euan Abercrombie,Gryffindors
4,99282,Sakura Akagi,Gryffindors


In [134]:
df.to_csv('character.csv')